In [1]:
import json
import boto3
import time


In [2]:
# Reading tweets
with open('../tweets.json','r',encoding='utf-8') as f:
    full_tweets = json.load(f)

In [3]:
# Create a kinesis client
client = boto3.client(
    'firehose',
    region_name='us-east-1',
    aws_access_key_id='Enter Your Access Key ID',
    aws_secret_access_key='Enter Your Secret Access Key',
)

c=0

In [4]:
# Preprocess tweet and sending them to firehose
batch_size = 500

for i in range(10500,len(full_tweets),batch_size):
    print(f'Start Next Batch From {i} to {i+batch_size}')
    time.sleep(1)
    batch_tweets = full_tweets[i:i+batch_size]
    
    batch_request = []
    print('Loading',end='')
    dot=0
    for tweet in batch_tweets:
        if dot in range(0,500,100):
            print('.',end='')
            time.sleep(1)
            
        # Unpacking as it is unreadable by quicksight
        tweet['lat'],tweet['lon'] = tweet['tweet_coord']
        del tweet['tweet_coord']
        
        # Encoding tweet string to bytes.
        tweet_payload = json.dumps(tweet) + '\n'
        encoded_tweet_payload = tweet_payload.encode('utf-8')
        batch_request.append({'Data':encoded_tweet_payload})
        dot+=1
        
    print(' Done!')
    print('Sending Data to Firehose...')
    response = client.put_record_batch(
            DeliveryStreamName = "Tweets-Ingest",
            Records = batch_request)
    
    
    print(f'Batch Number {c} Sent Successfully.')
    print('-'*20)
    time.sleep(5)
    c+=1


Start Next Batch From 0 to 500
...... Done!
Sending Data to Firehose...
Batch Number 0 Sent Successfully.
--------------------
Start Next Batch From 500 to 1000
...... Done!
Sending Data to Firehose...
Batch Number 1 Sent Successfully.
--------------------
